# Census Data Processing
This notebook is dedicated to aggregate census data based off zipcode.

## Process Poverty Data From Census Data

In [88]:
import pandas as pd
import numpy as np
from IPython.display import display

# import zipcode data
df_zip = pd.read_csv("../data/zipcodes/zip_code_database.csv")
ma_zip_df = df_zip[df_zip["state"] == "MA"]

# Ensure the zip code column in the zipcode DataFrame has 5 digits
ma_zip_df['zip'] = ma_zip_df['zip'].astype(str).str.zfill(5)

# import poverty data
df_poverty = pd.read_csv("../data/census/poverty_status_sex_age.csv")

# Also, ensure the poverty DataFrame’s ZCTA5A column is a 5-digit string
df_poverty['ZCTA5A'] = df_poverty['ZCTA5A'].astype(str).str.zfill(5)

# Merge the two DataFrames using an inner join so that only matching ZIP codes are retained
df_joined = pd.merge(ma_zip_df, df_poverty, left_on='zip', right_on='ZCTA5A', how='inner')

# Select the relevant columns
df_final = df_joined[['state', 'zip', 'AQ5ZE001', 'AQ5ZE002', 'AQ5ZE031']]
df_final.rename(columns={
    'AQ5ZE001': 'total_population',
    'AQ5ZE002': 'below_poverty_level',
    'AQ5ZE031': 'at_or_above_poverty_level'
}, inplace=True)
df_final['poverty_rate'] = df_final['below_poverty_level'].astype(float) / df_final['total_population'].astype(float)
df_final = df_final[['state', 'zip', 'poverty_rate']]

display(df_final)

/var/folders/bb/28yzn45x2_s9s2kv375_g8r00000gn/T/ipykernel_22688/572441688.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ma_zip_df['zip'] = ma_zip_df['zip'].astype(str).str.zfill(5)
/var/folders/bb/28yzn45x2_s9s2kv375_g8r00000gn/T/ipykernel_22688/572441688.py:13: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,35,36,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146

,state,zip,poverty_rate
0,MA,01001,0.071364
1,MA,01002,0.221961
2,MA,01003,0.065217
3,MA,01005,0.088889
4,MA,01007,0.082552
...,...,...,...
534,MA,02777,0.031950
535,MA,02779,0.012743
536,MA,02780,0.129964
537,MA,02790,0.071111


## Process Median Household Income Data 

In [89]:
df_median_income = pd.read_csv("../data/census/median_household_income.csv")
df_median_income['ZCTA5A'] = df_median_income['ZCTA5A'].astype(str).str.zfill(5)

df_median_income = df_median_income[['ZCTA5A', 'AS9DE001']]
df_median_income.rename(columns={
    'ZCTA5A': 'zip',
    'AS9DE001': 'median_income_household'
}, inplace=True)

# Merge the poverty DataFrame with the median income DataFrame
df_final = pd.merge(df_final, df_median_income, left_on='zip', right_on='zip', how='inner')

df_final.sort_values(by='median_income_household', ascending=True, inplace=True)
display(df_final)


/var/folders/bb/28yzn45x2_s9s2kv375_g8r00000gn/T/ipykernel_22688/2006241279.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,22,23,24,25,26,27,28,29,30,31,32,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,54,55,56,57,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df_median_income = pd.read_csv("../data/census/median_household_income.csv")


,state,zip,poverty_rate,median_income_household
190,MA,01561,NaN,-666666666
101,MA,01260,0.000000,-666666666
493,MA,02663,0.000000,-666666666
487,MA,02652,0.083333,-666666666
36,MA,01066,0.619403,-666666666
...,...,...,...,...
4,MA,01007,0.082552,99056
16,MA,01029,0.026565,99188
177,MA,01531,0.130081,99375
122,MA,01351,0.074061,99375


## Process Educational Attainment

In [90]:
df_edu = pd.read_csv("../data/census/education_snap_per_capita_income.csv", skiprows=[1], header=0)
df_edu['ZCTA5A'] = df_edu['ZCTA5A'].astype(str).str.zfill(5)

# Group 1: High school education and below
group1_cols = ["ASP3E002", "ASP3E003", "ASP3E004", "ASP3E005", "ASP3E006",
               "ASP3E007", "ASP3E008", "ASP3E009", "ASP3E010", "ASP3E011",
               "ASP3E012", "ASP3E013", "ASP3E014", "ASP3E015", "ASP3E016",
               "ASP3E017", "ASP3E018"]
df_edu[group1_cols] = df_edu[group1_cols].astype(float).fillna(0)

# Group 2: College and above, including some college
group2_cols = ["ASP3E019", "ASP3E020", "ASP3E021", "ASP3E022", 
               "ASP3E023", "ASP3E024", "ASP3E025"]
df_edu[group2_cols] = df_edu[group2_cols].astype(float).fillna(0)

# Sum the counts for each group
df_edu["low_ed_sum"] = df_edu[group1_cols].sum(axis=1)
df_edu["high_ed_sum"] = df_edu[group2_cols].sum(axis=1)

# Compute percentages based on total population aged 25 and over (ASP3E001)
df_edu["percent_lower_education"] = df_edu["low_ed_sum"] / df_edu["ASP3E001"]
df_edu["percent_higher_education"] = df_edu["high_ed_sum"] / df_edu["ASP3E001"]

# Only select the relevant columns
df_edu_final = df_edu[["ZCTA5A", "percent_lower_education", "percent_higher_education"]]
df_edu_final.rename(columns={"ZCTA5A": "zip"}, inplace=True)

# Merge the education DataFrame with the poverty DataFrame
df_final = pd.merge(df_final, df_edu_final, left_on='zip', right_on='zip', how='inner')

display(df_final)

/var/folders/bb/28yzn45x2_s9s2kv375_g8r00000gn/T/ipykernel_22688/2399464882.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edu_final.rename(columns={"ZCTA5A": "zip"}, inplace=True)


,state,zip,poverty_rate,median_income_household,percent_lower_education,percent_higher_education
0,MA,01561,NaN,-666666666,NaN,NaN
1,MA,01260,0.000000,-666666666,1.000000,0.000000
2,MA,02663,0.000000,-666666666,0.065041,0.934959
3,MA,02652,0.083333,-666666666,0.491342,0.508658
4,MA,01066,0.619403,-666666666,0.629630,0.370370
...,...,...,...,...,...,...
534,MA,01007,0.082552,99056,0.250489,0.749511
535,MA,01029,0.026565,99188,0.432638,0.567362
536,MA,01531,0.130081,99375,0.341292,0.658708
537,MA,01351,0.074061,99375,0.319862,0.680138


## Process Food Stamps / SNAP Per Household

In [91]:
df_snap = pd.read_csv("../data/census/education_snap_per_capita_income.csv", skiprows=[1], header=0)
df_snap['ZCTA5A'] = df_snap['ZCTA5A'].astype(str).str.zfill(5)

# Calculate the percentage of SNAP participation
df_snap['percent_snap_participation'] = df_snap['ASRAE002'].astype(float) / df_snap['ASRAE001'].astype(float)

# Rename the columns for clarity
df_snap.rename(columns={
    'ZCTA5A': 'zip',
}, inplace=True)

# Select relevant columns
df_snap = df_snap[['zip', 'percent_snap_participation']]

# Merge the SNAP DataFrame with the final DataFrame
df_final = pd.merge(df_final, df_snap, left_on='zip', right_on='zip', how='inner')

display(df_final)

,state,zip,poverty_rate,median_income_household,percent_lower_education,percent_higher_education,percent_snap_participation
0,MA,01561,NaN,-666666666,NaN,NaN,NaN
1,MA,01260,0.000000,-666666666,1.000000,0.000000,0.000000
2,MA,02663,0.000000,-666666666,0.065041,0.934959,0.000000
3,MA,02652,0.083333,-666666666,0.491342,0.508658,0.063863
4,MA,01066,0.619403,-666666666,0.629630,0.370370,0.433333
...,...,...,...,...,...,...,...
534,MA,01007,0.082552,99056,0.250489,0.749511,0.103749
535,MA,01029,0.026565,99188,0.432638,0.567362,0.088525
536,MA,01531,0.130081,99375,0.341292,0.658708,0.047368
537,MA,01351,0.074061,99375,0.319862,0.680138,0.165331


## Process Per Capita Income

In [92]:
df_capita_income = pd.read_csv("../data/census/education_snap_per_capita_income.csv", skiprows=[1], header=0)
df_capita_income['ZCTA5A'] = df_capita_income['ZCTA5A'].astype(str).str.zfill(5)

df_capita_income = df_capita_income[['ZCTA5A', 'ASRTE001']]
df_capita_income.rename(columns={
    'ZCTA5A': 'zip',
    'ASRTE001': 'per_capita_income'
}, inplace=True)

# Merge the per capita income DataFrame with the final DataFrame
df_final = pd.merge(df_final, df_capita_income, left_on='zip', right_on='zip', how='inner')

display(df_final)


,state,zip,poverty_rate,median_income_household,percent_lower_education,percent_higher_education,percent_snap_participation,per_capita_income
0,MA,01561,NaN,-666666666,NaN,NaN,NaN,-666666666.0
1,MA,01260,0.000000,-666666666,1.000000,0.000000,0.000000,NaN
2,MA,02663,0.000000,-666666666,0.065041,0.934959,0.000000,104119.0
3,MA,02652,0.083333,-666666666,0.491342,0.508658,0.063863,91153.0
4,MA,01066,0.619403,-666666666,0.629630,0.370370,0.433333,16546.0
...,...,...,...,...,...,...,...,...
534,MA,01007,0.082552,99056,0.250489,0.749511,0.103749,46109.0
535,MA,01029,0.026565,99188,0.432638,0.567362,0.088525,67302.0
536,MA,01531,0.130081,99375,0.341292,0.658708,0.047368,46536.0
537,MA,01351,0.074061,99375,0.319862,0.680138,0.165331,38954.0


## Clean Dataframe

In [93]:
# Convert key numeric columns to floats
numeric_cols = [
    "poverty_rate", "median_income_household",
    "percent_lower_education", "percent_higher_education",
    "percent_snap_participation", "per_capita_income"
]
for col in numeric_cols:
    df_final[col] = pd.to_numeric(df_final[col], errors="coerce")

# Replace specific invalid values with NaN
df_final.replace([-666666666, -222222222], np.nan, inplace=True)

# Ensure zip codes are 5 digits
df_final['zip'] = df_final['zip'].astype(str).str.zfill(5)

# Want to see how many rows have NaN or invalid percentage values (0.0 or 1.0)
bad = df_final[numeric_cols].isna() | (df_final[numeric_cols] == 0.0) | (df_final[numeric_cols] == 1.0)

# Compute the ratio of bad values per row
df_final['missing_ratio'] = bad.mean(axis=1)

# Drop rows where more than 50% of numeric values are missing
threshold = 0.5
df_clean = df_final[df_final['missing_ratio'] <= threshold].copy()

# For the remaining rows, fill bad values with the median of the respective column
for col in numeric_cols:
    median_val = df_clean[col].median()
    df_clean[col].replace(0.0, np.nan, inplace=True)
    df_clean[col].replace(1.0, np.nan, inplace=True)
    df_clean[col].fillna(median_val, inplace=True)

# Drop the helper column
df_clean.drop(columns=['missing_ratio'], inplace=True)

display(df_clean)

# Save the final DataFrame to a CSV file
df_clean.to_csv("../data/census/massachusetts_census.csv", index=False)

/var/folders/bb/28yzn45x2_s9s2kv375_g8r00000gn/T/ipykernel_22688/1081609771.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean[col].replace(0.0, np.nan, inplace=True)
/var/folders/bb/28yzn45x2_s9s2kv375_g8r00000gn/T/ipykernel_22688/1081609771.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values al

,state,zip,poverty_rate,median_income_household,percent_lower_education,percent_higher_education,percent_snap_participation,per_capita_income
2,MA,02663,0.067579,107024.5,0.065041,0.934959,0.091044,104119.0
3,MA,02652,0.083333,107024.5,0.491342,0.508658,0.063863,91153.0
4,MA,01066,0.619403,107024.5,0.629630,0.370370,0.433333,16546.0
5,MA,02647,0.067579,107024.5,0.128492,0.871508,0.091044,82214.0
10,MA,02641,0.067579,107024.5,0.122951,0.877049,0.091044,73811.0
...,...,...,...,...,...,...,...,...
534,MA,01007,0.082552,99056.0,0.250489,0.749511,0.103749,46109.0
535,MA,01029,0.026565,99188.0,0.432638,0.567362,0.088525,67302.0
536,MA,01531,0.130081,99375.0,0.341292,0.658708,0.047368,46536.0
537,MA,01351,0.074061,99375.0,0.319862,0.680138,0.165331,38954.0
